# Прогнозирование температуры плавления стали

**Описание проекта**

Чтобы оптимизировать производственные расходы, металлургический комбинат ООО «Так закаляем сталь» решил уменьшить потребление электроэнергии на этапе обработки стали. Вам предстоит построить модель, которая предскажет температуру стали.

**Описание этапа обработки стали**

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены в крышке ковша.

Из сплава выводится сера (десульфурация), добавлением примесей корректируется химический состав и отбираются пробы. Сталь легируют — изменяют её состав — подавая куски сплава из бункера для сыпучих материалов или проволоку через специальный трайб-аппарат (англ. tribe, «масса»).

Перед тем как первый раз ввести легирующие добавки, измеряют температуру стали и производят её химический анализ. Потом температуру на несколько минут повышают, добавляют легирующие материалы и продувают сплав инертным газом. Затем его перемешивают и снова проводят измерения. Такой цикл повторяется до достижения целевого химического состава и оптимальной температуры плавки.

Тогда расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (англ. slab, «плита»).

**Описание данных**

Данные состоят из файлов, полученных из разных источников:

1. `data_arc.csv` — данные об электродах;
2. `data_bulk.csv` — данные о подаче сыпучих материалов (объём);
3. `data_bulk_time.csv` — данные о подаче сыпучих материалов (время);
4. `data_gas.csv` — данные о продувке сплава газом;
5. `data_temp.csv` — результаты измерения температуры;
6. `data_wire.csv` — данные о проволочных материалах (объём);
7. `data_wire_time.csv` — данные о проволочных материалах (время).

Во всех файлах столбец `key` содержит номер партии. В файлах может быть несколько строк с одинаковым значением `key`: они соответствуют разным итерациям обработки.

**План работы**

1. **Предобработка и исследовательский анализ данных**. Проверить наличие пропусков, некорректных значений, дубликатов, провести их обработку, привести данные к нужному типу. Изучить признаки, их основные статистики, построить графики при необходимости. Удалить признаки, не несущие смысловой нагрузки или явлющиеся утечкой таргета. Подготовить итоговую таблицу "объектов-признаков", где объектами являются уникальные ключи партий.
   
2. **Обучение моделей**. Разделить выборку на обучающую и тестовую. Исследовать разные ML модели. Выбрать признаковое пространство, модель и ее гиперпараметры, имеющее лучшее качество на кросс-валидации
   
3. **Оценка эффективности моделей на тестовой выборке**. При недостижении целевого значения метрики качества вернуться к этапу 2, сформировать дополнительные признаки, продолжить выбор моделей и настройку гиперпараметров.

In [1]:
import pandas as pd
import numpy as np
import optuna
import plotly.express as px

from caseconverter import snakecase
from collections import defaultdict
from IPython.display import display

from ydata_profiling import ProfileReport
from fast_ml import eda

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from category_encoders import MEstimateEncoder

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error

In [2]:
FIG_WIDTH = 9 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [14]:
try:
    raw = {
        'arc': pd.read_csv('data_arc.csv'),
        'bulk_t': pd.read_csv('data_bulk_time.csv'),
        'bulk': pd.read_csv('data_bulk.csv'),
        'gas': pd.read_csv('data_gas.csv'),
        'temp': pd.read_csv('data_temp.csv'),
        'wire_t': pd.read_csv('data_wire_time.csv'),
        'wire': pd.read_csv('data_wire.csv'),
    }
except:
    raw = {
        'arc': pd.read_csv('/datasets/data_arc.csv'),
        'bulk_t': pd.read_csv('/datasets/data_bulk_time.csv'),
        'bulk': pd.read_csv('/datasets/data_bulk.csv'),
        'gas': pd.read_csv('/datasets/data_gas.csv'),
        'temp': pd.read_csv('/datasets/data_temp.csv'),
        'wire_t': pd.read_csv('/datasets/data_wire_time.csv'),
        'wire': pd.read_csv('/datasets/data_wire.csv'),
    }

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [24]:
for dataset in raw.keys():
    print('#' * 50, 'Summary for', dataset, 'dataset', '#' * 50)
    display(eda.df_info(raw[dataset]))

################################################## Summary for arc dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3214,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Начало нагрева дугой,object,Categorical,14875,"[2019-05-03 11:02:14, 2019-05-03 11:07:28, 201...",0,0.0
Конец нагрева дугой,object,Categorical,14876,"[2019-05-03 11:06:02, 2019-05-03 11:10:33, 201...",0,0.0
Активная мощность,float64,Numerical,14869,"[0.9760587506, 0.8056070805, 0.7443634272, 1.6...",0,0.0
Реактивная мощность,float64,Numerical,14866,"[0.6870837212, 0.5202852488, 0.4988054037, 1.0...",0,0.0


################################################## Summary for bulk_t dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3129,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Bulk 1,object,Categorical,252,"[nan, 2019-05-03 17:50:19, 2019-05-05 10:09:13...",2877,91.946309
Bulk 2,object,Categorical,22,"[nan, 2019-05-07 08:36:19, 2019-05-07 10:05:35...",3107,99.2969
Bulk 3,object,Categorical,1298,"[nan, 2019-05-03 20:13:36, 2019-05-04 03:47:10...",1831,58.517098
Bulk 4,object,Categorical,1014,"[2019-05-03 11:21:30, 2019-05-03 11:46:38, 201...",2115,67.59348
Bulk 5,object,Categorical,77,"[nan, 2019-05-07 08:36:19, 2019-05-07 10:05:35...",3052,97.53915
Bulk 6,object,Categorical,576,"[nan, 2019-05-03 18:52:57, 2019-05-03 21:15:20...",2553,81.591563
Bulk 7,object,Categorical,25,"[nan, 2019-05-07 11:37:42, 2019-05-15 09:35:53...",3104,99.201023
Bulk 8,object,Categorical,1,"[nan, 2019-07-05 17:46:11]",3128,99.968041
Bulk 9,object,Categorical,19,"[nan, 2019-05-14 00:04:56, 2019-05-14 00:37:47...",3110,99.392777


################################################## Summary for bulk dataset ##################################################


...

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3216,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Время замера,object,Categorical,15907,"[2019-05-03 11:16:18, 2019-05-03 11:25:53, 201...",0,0.0
Температура,float64,Numerical,175,"[1571.0, 1604.0, 1618.0, 1601.0, 1613.0, 1581....",2901,18.237254


################################################## Summary for wire_t dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3081,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Wire 1,object,Categorical,3055,"[2019-05-03 11:11:41, 2019-05-03 11:46:10, 201...",26,0.843882
Wire 2,object,Categorical,1079,"[nan, 2019-05-03 13:32:06, 2019-05-03 13:59:54...",2002,64.978903
Wire 3,object,Categorical,63,"[nan, 2019-05-04 04:47:53, 2019-05-04 05:40:08...",3018,97.955209
Wire 4,object,Categorical,14,"[nan, 2019-05-07 08:48:23, 2019-05-07 10:14:55...",3067,99.545602
Wire 5,object,Categorical,1,"[nan, 2019-08-08 16:01:07]",3080,99.967543
Wire 6,object,Categorical,73,"[nan, 2019-05-07 08:37:20, 2019-05-07 10:06:35...",3008,97.630639
Wire 7,object,Categorical,11,"[nan, 2019-07-23 12:42:00, 2019-07-23 12:08:44...",3070,99.642973
Wire 8,object,Categorical,19,"[nan, 2019-05-14 00:03:32, 2019-05-14 00:36:39...",3062,99.383317
Wire 9,object,Categorical,29,"[nan, 2019-05-04 16:09:11, 2019-06-01 02:25:26...",3052,99.058747


################################################## Summary for wire dataset ##################################################


,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
key,int64,Numerical,3081,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0,0.0
Wire 1,float64,Numerical,2251,"[60.059998, 96.052315, 91.160157, 89.063515, 8...",26,0.843882
Wire 2,float64,Numerical,713,"[nan, 9.11456, 9.143681, 12.376, 52.416003, 37...",2002,64.978903
Wire 3,float64,Numerical,56,"[nan, 250.041794, 276.324066, 96.047951, 88.64...",3018,97.955209
Wire 4,float64,Numerical,14,"[nan, 24.148801, 43.065361, 33.182243, 103.034...",3067,99.545602
Wire 5,float64,Numerical,1,"[nan, 15.132]",3080,99.967543
Wire 6,float64,Numerical,69,"[nan, 60.094322000000005, 54.122643, 64.212723...",3008,97.630639
Wire 7,float64,Numerical,10,"[nan, 5.035472, 8.49004, 1.053936, 0.234208, 1...",3070,99.642973
Wire 8,float64,Numerical,13,"[nan, 46.002319, 46.094879, 46.187439000000005...",3062,99.383317
Wire 9,float64,Numerical,25,"[nan, 35.0532, 4.622800000000001, 35.0168, 10....",3052,99.058747


Числовые распределения:

И детальный отчет: